In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

# Pixel coordinates-only net

In [ ]:
# Build model
dropout = 0.3
num_consec = 5
input_layer = Input(shape=(num_consec * 2,))
x = Dense(
    128, 
    activation='relu',
    kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=1e-7)
)(input_layer)
# x = Dropout(dropout)(x)
x = Dense(
    64, 
    activation='relu',
#     kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=1e-7)
)(x)
x = Dense(
    32, 
    activation='relu',
#     kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=1e-7)
)(x)
x = Dense(
    16, 
    activation='relu',
#     kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=1e-7)
)(x)
output_layer = Dense(
    1, 
    activation='sigmoid',
#     kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=1e-7)
)(x)

model = Model(input_layer, output_layer)

In [ ]:
opt = optimizers.Adam(lr=1e-6)
model.compile(
    optimizer='adam', 
    loss="binary_crossentropy",
    metrics=['accuracy']
)
model.summary()

In [ ]:
def diffuse(x):
    for i in range(x.shape[0]):
        val = 0
        if i > 0:
            val += x[i-1]
        if i < x.shape[0]-1:
            val += x[i+1]
        x[i] = max(x[i], val/2.)
    return x

In [ ]:
matches = list('match' + str(i) for i in range(1, 23))

x_train, y_train = [], []
for match in matches:
    for video in os.listdir('/home/code-base/scratch_space/data/%s/rally_video/' % match):
        rally, _ = os.path.splitext(video)
        trajectory = pd.read_csv('/home/code-base/scratch_space/data/%s/ball_trajectory/%s_ball.csv' % (match, rally))
        hit = pd.read_csv('/home/code-base/scratch_space/data/%s/shot/%s_hit.csv' % (match, rally))
        trajectory[trajectory.X == 0] = float('nan')
        trajectory[trajectory.Y == 0] = float('nan')
        
        trajectory = trajectory.assign(X=trajectory.X.interpolate(method='slinear'))
        trajectory = trajectory.assign(Y=trajectory.Y.interpolate(method='slinear'))
        trajectory.fillna(method='bfill', inplace=True)
        trajectory.fillna(method='ffill', inplace=True)
        
        x_list, y_list = [], []
        for i in range(num_consec):
            end = min(len(trajectory.X), len(hit.hit))-num_consec+i+1
            x = np.array(list(zip(trajectory.X[i:end], trajectory.Y[i:end])))
            y = hit.hit[i:end].to_numpy().astype(float)
            x_list.append(x)
            y_list.append(y)
        
        x_t = np.hstack(x_list)
        y_t = np.sum(np.column_stack(y_list[1:-1]), axis=1)
        x_train.append(x_t)
        y_train.append(y_t)

x_train = np.vstack(x_train)
y_train = np.hstack(y_train)

Xb, Yb = trajectory.X.tolist(), trajectory.Y.tolist()
plt.plot(range(400), Yb[:400])
plt.plot(range(400), Xb[:400])
plt.show()

In [ ]:
import ruptures as rpt
rpt.display(trajectory.Y, hit.index[hit.hit == 1].tolist(), hit.index[hit.hit == 1].tolist())
rpt.display(trajectory.X, hit.index[hit.hit == 1].tolist(), hit.index[hit.hit == 1].tolist())

plt.show()

In [ ]:
import random

p = sum(y_train == 1) / sum(y_train == 0)
subsample = []
for i in range(x_train.shape[0]):
    if random.random() < p or y_train[i]:
        subsample.append(i)

x_sub = x_train[subsample]
y_sub = y_train[subsample]

In [ ]:
from sklearn.utils import class_weight
from sklearn.metrics import classification_report

# cw = class_weight.compute_class_weight('balanced', [0,1], y_train)
model.fit(
    x_sub, y_sub, 
    batch_size=64, 
    epochs=1000,
#     class_weight={0: cw[0] / sum(cw), 1: 1.5 * cw[1] / sum(cw)},
    shuffle=True,
    validation_split=0.1,
    verbose=1)

In [ ]:
y_pred = np.round(model.predict(x_train) > 0.9)
print(classification_report(y_train, y_pred))

In [ ]:
import keras
keras.models.save_model(model, './hitnet_weights')

# Incorporating pose and court information
## (Generating and separating pose into top and bottom)

In [1]:
from ai_badminton.pose import Pose, read_player_poses, process_pose_file
from ai_badminton.court import Court, read_court
from ai_badminton.trajectory import Trajectory
from ai_badminton.video_annotator import annotate_video

In [3]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

match = 'match21'
rally = '1_02_01'

import os
matches = list('match' + str(i) for i in range(1, 23))

for match in matches:
    for video in os.listdir('/home/code-base/scratch_space/data/%s/rally_video/' % match):
        rally, _ = os.path.splitext(video)
        
        court_pts = read_court('/home/code-base/scratch_space/data/%s/court/%s.out' % (match, rally))
        corners = [court_pts[1], court_pts[2], court_pts[0], court_pts[3]]
    
        def draw_court(corners):
            # Draws the court once we have the three corners
            court = Court(corners)
            frames[-1] = court.draw_lines(frames[-1])
            plt.imshow(frames[-1])
            plt.title(match)
   
        cap = cv2.VideoCapture('/home/code-base/scratch_space/data/%s/rally_video/%s.mp4' % (match, rally))
        if cap.isOpened() is False:
            print('Error opening video stream or file')

        ret, frame = cap.read()
        fig = plt.figure()
        frames = [frame]

        draw_court(corners)
        break
court = Court(corners)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-3-a0a7a1051503>:31: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
poses = process_pose_file(
    '/home/code-base/scratch_space/data/%s/poses/%s.out' % (match, rally), 
    '/home/code-base/scratch_space/data/%s/poses/%s' % (match, rally), 
    court,
    fullPose=True
)

Read in files. Processing poses...


  0%|          | 0/267586 [00:00<?, ?it/s]

Separating top and bottom poses...
Done!


In [8]:
import os
print(os.getcwd())

/trainman-mount/trainman-k8s-storage-6677a55e-8792-44ee-923d-9ed5edea05ca/ai-badminton/TrackNetv2/3_in_3_out


In [7]:
trajectory = Trajectory('/home/code-base/scratch_space/data/%s/ball_trajectory/%s_ball.csv' % (match, rally))
poses = read_player_poses('/home/code-base/scratch_space/data/%s/poses/%s' % (match, rally))
annotate_video(
    cap,
    court,
    poses,
    trajectory,
)

  0%|          | 0/646 [00:00<?, ?it/s]

In [5]:
import os
matches = list('match' + str(i) for i in range(1, 23))

for match in matches:
    for video in os.listdir('/home/code-base/scratch_space/data/%s/rally_video/' % match):
        rally, _ = os.path.splitext(video)
        
        court_pts = read_court('/home/code-base/scratch_space/data/%s/court/%s.out' % (match, rally))
        corners = [court_pts[1], court_pts[2], court_pts[0], court_pts[3]]
        court = Court(corners)

        print(match, rally)
        poses = process_pose_file(
            '/home/code-base/scratch_space/data/%s/poses/%s.out' % (match, rally), 
            '/home/code-base/scratch_space/data/%s/poses/%s' % (match, rally), 
            court,
            fullPose=True
        )

match1 1_02_00
Read in files. Processing poses...


  0%|          | 0/152150 [00:00<?, ?it/s]

Separating top and bottom poses...
Done!
match1 1_02_04
Read in files. Processing poses...


  0%|          | 0/49402 [00:00<?, ?it/s]

Separating top and bottom poses...
Done!
match1 1_06_08
Read in files. Processing poses...


...

Separating top and bottom poses...
Done!
match22 2_18_18
Read in files. Processing poses...


  0%|          | 0/227675 [00:00<?, ?it/s]

Separating top and bottom poses...
Done!
match22 1_07_02
Read in files. Processing poses...


  0%|          | 0/132474 [00:00<?, ?it/s]

Separating top and bottom poses...
Done!


# Creating a neural net that uses pose and court information

In [ ]:
import os
import pandas as pd
import numpy as np
import cv2

def read_court(filename):
    file = open(filename, 'r')
    coordinates = [[float(x) for x in line.split(';')] for line in file]
    return coordinates

num_consec = 5
matches = list('match' + str(i) for i in range(1, 23))

from sklearn.preprocessing import MinMaxScaler, StandardScaler

x_train, y_train = [], []
for match in matches:
    for video in os.listdir('/home/code-base/scratch_space/data/%s/rally_video/' % match):
        cap = cv2.VideoCapture('/home/code-base/scratch_space/data/%s/rally_video/%s' % (match, video))
        shape = cap.read()[1].shape
        height, width = shape[1], shape[0]

        rally, _ = os.path.splitext(video)
        trajectory = pd.read_csv('/home/code-base/scratch_space/data/%s/ball_trajectory/%s_ball.csv' % (match, rally))
        hit = pd.read_csv('/home/code-base/scratch_space/data/%s/shot/%s_hit.csv' % (match, rally))
        trajectory[trajectory.X == 0] = float('nan')
        trajectory[trajectory.Y == 0] = float('nan')
        
        trajectory = trajectory.assign(X=trajectory.X.interpolate(method='slinear'))
        trajectory = trajectory.assign(Y=trajectory.Y.interpolate(method='slinear'))
        trajectory.fillna(method='bfill', inplace=True)
        trajectory.fillna(method='ffill', inplace=True)
        
        bottom_player = pd.read_csv('/home/code-base/scratch_space/data/%s/poses/%s_bottom.csv' % (match, rally))
        top_player = pd.read_csv('/home/code-base/scratch_space/data/%s/poses/%s_top.csv' % (match, rally))
        bottom_player.drop('frame', axis=1, inplace=True)
        top_player.drop('frame', axis=1, inplace=True)
        bottom_player.fillna(0, inplace=True)
        top_player.fillna(0, inplace=True)
        
        x_list, y_list = [], []
        court_pts = read_court('/home/code-base/scratch_space/data/%s/court/%s.out' % (match, rally))
        corners = np.array([court_pts[1], court_pts[2], court_pts[0], court_pts[3]]).flatten()
    
        def reflect(x):
            x_ = np.array(x)
            for i in range(0, x.shape[1], 2):
                x_[:, i] = width - x_[:, i]
            return x_

        for i in range(num_consec):
            end = min(len(trajectory.X), len(hit.hit))-num_consec+i+1
            x_bird = np.array(list(zip(trajectory.X[i:end], trajectory.Y[i:end])))
            
            # Reduce amount of data by averaging coordinate values
#             bottom_x = np.median(bottom_player.values[i:end][:, ::2], axis=1)
#             bottom_y = np.median(bottom_player.values[i:end][:, 1::2], axis=1)
#             top_x = np.median(top_player.values[i:end][:, ::2], axis=1)
#             top_y = np.median(top_player.values[i:end][:, 1::2], axis=1)
            
            # Use median values only
#             x_pose = np.vstack([bottom_x, bottom_y, top_x, top_y]).T

            # Use entire pose
            x_pose = np.hstack([bottom_player.values[i:end], top_player.values[i:end]])
            x = np.hstack([x_bird, x_pose, np.array([corners for j in range(i, end)])])
#             x = np.hstack([x_bird, np.array([corners for j in range(i, end)])])
            y = hit.hit[i:end].to_numpy().astype(float)
            x_list.append(x)
            y_list.append(y)

        x_t = np.hstack(x_list)
        y_t = np.sum(np.column_stack(y_list[1:-1]), axis=1)
        
#         scaler = MinMaxScaler()
#         scaler.fit(x_t)
#         x_t = scaler.transform(x_t)
#         x_train.append(x_t)
#         y_train.append(y_t)
        
#         scaler = MinMaxScaler()
#         scaler.fit(reflect(x_t))
#         x_t = scaler.transform(reflect(x_t))
#         x_train.append(x_t)
#         y_train.append(y_t)

        x_train.append(x_t)
        y_train.append(y_t)
        x_train.append(reflect(x_t))
        y_train.append(y_t)

x_train = np.vstack(x_train)
y_train = np.hstack(y_train)

In [ ]:
from keras.models import *
from keras.layers import *
import keras.backend as K
from keras import optimizers
from keras.activations import *
import tensorflow as tf
import cv2
import math

dropout = 0.4

input_layer = Input(shape=(x_train[0].shape[0],))
x = Dense(
    70, 
    activation='relu',
#     kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=1e-9)
)(input_layer)
# x = ActivityRegularization(l1=1e-9)(x)
# x = BatchNormalization()(x)
# x = Dropout(dropout)(x)
x = Dense(
    30, 
    activation='relu',
#     kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=1e-9)
)(x)
# x = Dropout(dropout)(x)

# x = ActivityRegularization(l1=1e-9)(x)
# x = ActivityRegularization()(x)
# x = BatchNormalization()(x)

# x = Dense(
#     32, 
#     activation='relu',
# #     kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=1e-7)
# )(x)
# # x = ActivityRegularization(l1=1e-11)(x)
# x = BatchNormalization()(x)

# x = Dense(
#     16, 
#     activation='relu',
# #     kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=1e-7)
# )(x)
# x = BatchNormalization()(x)

output_layer = Dense(
    1, 
    activation='sigmoid',
#     kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=1e-7)
)(x)

model = Model(input_layer, output_layer)

In [ ]:
opt = optimizers.Adam(lr=1e-1)
model.compile(
    optimizer='adam', 
    loss="binary_crossentropy",
    metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()]
)
model.summary()

In [ ]:
import random

p = sum(y_train == 1) / sum(y_train == 0)
subsample = []
for i in range(x_train.shape[0]):
    if random.random() < p or y_train[i]:
        subsample.append(i)

x_sub = x_train[subsample]
y_sub = y_train[subsample]

from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(x_sub, y_sub, test_size=0.1, shuffle=True)

In [ ]:
from sklearn.utils import class_weight
from sklearn.metrics import classification_report

# from keras import backend as K
# K.set_value(model.optimizer.learning_rate, 1e-5)

model.fit(
    x_tr, y_tr, 
    batch_size=64, 
    epochs=1000,
    shuffle=True,
    validation_data=(x_val, y_val),
    verbose=1)

In [ ]:
y_pred = np.round(model.predict(x_train) > 0.7)
print(classification_report(y_train, y_pred))

In [ ]:
import keras
keras.models.save_model(model, './hitnet_plus_court_model')

# Conv. nets for shuttle detection

In [11]:
from keras import objectives, backend as K
import keras
import tensorflow as tf

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(
    device_count={ 'CPU': 16 , 'GPU':4},
    inter_op_parallelism_threads=0,
    allow_soft_placement=True,
    gpu_options= {'allow_growth': True, 'visible_device_list': "0,1,2,3"},
    intra_op_parallelism_threads=0,
))

K.set_session(sess)

In [145]:
import tensorflow as tf
import cv2
import math

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
from tensorflow.keras import optimizers
from tensorflow.keras.activations import *
from tensorflow.keras import regularizers


dropout = 0.5
# mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1", "/gpu:2", "/gpu:3"])
# mirrored_strategy = tf.distribute.MirroredStrategy()

# with mirrored_strategy.scope():
input_layer = Input(shape=(x_train[0].shape[0],))
x = Reshape(target_shape=(num_consec, x_train[0].shape[0] // (2 * num_consec), 2, 1))(input_layer)
x = Conv3D(
    filters=10,
    kernel_size=(1, 39, 2),
    kernel_initializer='he_uniform',
    activation='relu',
    data_format="channels_last",
#     kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
#     bias_regularizer=regularizers.l2(1e-4),
#     activity_regularizer=regularizers.l2(1e-5)
)(x)
x_base = Reshape(target_shape=(num_consec, 10))(x)
x2 = Conv1D(
    filters=6,
    kernel_size=2,
    kernel_initializer='he_uniform',
    activation='relu',
#     kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
#     bias_regularizer=regularizers.l2(1e-4),
#     activity_regularizer=regularizers.l2(1e-5)
)(x_base)
x2 = Dense(
    6,
    activation='relu'
)(Flatten()(x2))

x3 = Conv1D(
    filters=4,
    kernel_size=3,
    kernel_initializer='he_uniform',
    activation='relu',
#     kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
#     bias_regularizer=regularizers.l2(1e-4),
#     activity_regularizer=regularizers.l2(1e-5)
)(x_base)
x3 = Dense(
    4,
    activation='relu'
)(Flatten()(x3))
x4 = Conv1D(
    filters=2,
    kernel_size=4,
    kernel_initializer='he_uniform',
    activation='relu',
#     kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
#     bias_regularizer=regularizers.l2(1e-4),
#     activity_regularizer=regularizers.l2(1e-5)
)(x_base)
x4 = Dense(
    2,
    activation='relu'
)(Flatten()(x4))

x = Concatenate()([x2, x3, x4])
x = Dense(
    12,
    activation='relu'
)(x)
output_layer = Dense(2)(x)

model = Model(input_layer, output_layer)

In [146]:
model.compile(
    optimizer='adam', 
    loss="mse",
    metrics=['mse']
)
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 780)]        0                                            
__________________________________________________________________________________________________
reshape_31 (Reshape)            (None, 10, 39, 2, 1) 0           input_17[0][0]                   
__________________________________________________________________________________________________
conv3d_15 (Conv3D)              (None, 10, 1, 1, 10) 790         reshape_31[0][0]                 
__________________________________________________________________________________________________
reshape_32 (Reshape)            (None, 10, 10)       0           conv3d_15[0][0]                  
___________________________________________________________________________________________

In [ ]:
# from tensorflow.keras import backend as K
# K.set_value(model.optimizer.learning_rate, 1e-4)

model.fit(
    x_train, y_train, 
    batch_size=64, 
    epochs=3000,
#     class_weight={0: 1, 1: 1, 2: 1, 3: 10},
    shuffle=True,
#     validation_data=(x_val, y_val),
    verbose=1)

Epoch 1/3000
6039/6039 [==============================] - 18s 3ms/step - loss: 1.8120e-04 - mse: 1.8120e-04
Epoch 2/3000
6039/6039 [==============================] - 19s 3ms/step - loss: 1.8098e-04 - mse: 1.8098e-04
Epoch 3/3000
6039/6039 [==============================] - 19s 3ms/step - loss: 1.7984e-04 - mse: 1.7984e-04
Epoch 4/3000
6039/6039 [==============================] - 19s 3ms/step - loss: 1.7689e-04 - mse: 1.7689e-04
Epoch 5/3000
6039/6039 [==============================] - 18s 3ms/step - loss: 1.7958e-04 - mse: 1.7958e-04
Epoch 6/3000
6039/6039 [==============================] - 19s 3ms/step - loss: 1.7902e-04 - mse: 1.7902e-04
Epoch 7/3000
6039/6039 [==============================] - 18s 3ms/step - loss: 1.7442e-04 - mse: 1.7442e-04
Epoch 8/3000
6039/6039 [==============================] - 18s 3ms/step - loss: 1.7269e-04 - mse: 1.7269e-04
Epoch 9/3000
6039/6039 [==============================] - 19s 3ms/step - loss: 1.7677e-04 - mse: 1.7677e-04
Epoch 10/3000
6039/6039 [===

In [67]:
os.chdir('/home/code-base/user_space/ai-badminton/TrackNetv2/3_in_3_out')

In [68]:
from tensorflow.python.keras.saving import hdf5_format
import h5py

with h5py.File('./hitnet_conv_model_shuttle_refinement.h5', mode='w') as f:
    hdf5_format.save_model_to_hdf5(model, f)

# Look at mis-predicted shots

In [100]:
os.getcwd()
os.chdir('/home/code-base/user_space/ai-badminton/notebooks')

In [110]:
outvid = cv2.VideoWriter('./output/mispredicted.mp4', cv2.VideoWriter_fourcc('M','P','4','V'), 10, (width, height))

for match in matches[:5]:
    for video in os.listdir('/home/code-base/scratch_space/data/%s/rally_video/' % match):
        cap = cv2.VideoCapture('/home/code-base/scratch_space/data/%s/rally_video/%s' % (match, video))
        shape = cap.read()[1].shape
        height, width = shape[0], shape[1]

        rally, _ = os.path.splitext(video)
        trajectory = pd.read_csv('/home/code-base/scratch_space/data/%s/ball_trajectory/%s_ball.csv' % (match, rally))
        hit = pd.read_csv('/home/code-base/scratch_space/data/%s/shot/%s_hit.csv' % (match, rally))
        trajectory[trajectory.X == 0] = float('nan')
        trajectory[trajectory.Y == 0] = float('nan')
        
        trajectory = trajectory.assign(X=trajectory.X.interpolate(method='slinear'))
        trajectory = trajectory.assign(Y=trajectory.Y.interpolate(method='slinear'))
        trajectory.fillna(method='bfill', inplace=True)
        trajectory.fillna(method='ffill', inplace=True)
        
        bottom_player = pd.read_csv('/home/code-base/scratch_space/data/%s/poses/%s_bottom.csv' % (match, rally))
        top_player = pd.read_csv('/home/code-base/scratch_space/data/%s/poses/%s_top.csv' % (match, rally))
        bottom_player.drop('frame', axis=1, inplace=True)
        top_player.drop('frame', axis=1, inplace=True)
        bottom_player.fillna(0, inplace=True)
        top_player.fillna(0, inplace=True)
        
        x_list, y_list = [], []
        court_pts = read_court('/home/code-base/scratch_space/data/%s/court/%s.out' % (match, rally))
        corners = np.array([court_pts[1], court_pts[2], court_pts[0], court_pts[3]]).flatten()

        # Identify first hit by distance to pose
        # and then alternate hits
        def dist_to_pose(pose, p):
            pose = pose.reshape(17, 2)
            p = p.reshape(1, 2)
            D = np.sum((pose - p) * (pose - p), axis=1)
            return min(D)
        
        y_new = np.array(hit.hit.to_numpy())
        last = -1
        for i in range(y_new.shape[0]):
            if not y_new[i]:
                continue
                
            if last == -1:
                p = np.array([trajectory.X[i], trajectory.Y[i]])
                db = dist_to_pose(bottom_player.values[i], p)
                dt = dist_to_pose(top_player.values[i], p)
                ratio = max((db/dt)**0.5, (dt/db)**0.5)
                
                if dt < db:
                    y_new[i] = 2
                else:
                    y_new[i] = 1
                last = y_new[i]
                
                # Bad cases are all 2's
                if 0.9 < ratio < 1.1:
                    y_new[i] = 1
                print('Check', match, video, last)
            else:
                y_new[i] = 3 - last
                last = y_new[i]
        
        for i in range(num_consec):
            end = min(len(trajectory.X), len(hit.hit))-num_consec+i+1
            x_bird = np.array(list(zip(trajectory.X[i:end], trajectory.Y[i:end])))

            # Use entire pose
            x_pose = np.hstack([bottom_player.values[i:end], top_player.values[i:end]])
            x = np.hstack([x_bird, x_pose, np.array([corners for j in range(i, end)])])

            y = y_new[i:end]
            x_list.append(x)
            y_list.append(y)

        x_t = np.hstack(x_list)
        y_t = np.max(np.column_stack(y_list[2:-2]), axis=1)
        y_pred = np.argmax(model.predict(x_t), axis=1)
        
        nframes = len(hit.hit)
        is_bad = [0] * nframes
        for i in range(y_t.shape[0]):
            if y_t[i] > 0 and y_t[i] != y_pred[i]:
                for j in range(max(0, i-20), min(nframes, i+20)):
                    if abs(j-i) < 4:
                        if y_pred[i] > 0:
                            is_bad[j] = 3
                        else:
                            is_bad[j] = 2
                    elif is_bad[j] == 0:
                        is_bad[j] = 1
        
        for i in range(nframes):
            if is_bad[i]:
                cap.set(cv2.CAP_PROP_POS_FRAMES, i)
                ret, frame = cap.read()
                if is_bad[i] == 3:
                    frame[-50:, -50:] = np.array([255, 0, 0])
                elif is_bad[i] == 2:
                    frame[-50:, -50:] = np.array([0, 0, 255])
                    
                centre = (int(trajectory.X[i]), int(trajectory.Y[i]))
                radius = 5
                colour = (0, 255, 0)
                thickness = -1
                frame = cv2.circle(frame, centre, radius, colour, thickness)
                
                players = [bottom_player.values, top_player.values]
                for j in range(2):
                    xy = players[j][i]
                    pose = Pose()
                    pose.init_from_kparray(xy)
                    player_poses.append(pose)
                    frame = pose.draw_skeleton(frame, colour=(128, 128 + j * 127, 128))
                outvid.write(frame)
outvid.release()

Check match1 1_01_00.mp4 2
Check match1 1_03_06.mp4 1
Check match1 1_02_04.mp4 2
Check match1 1_02_03.mp4 1
Check match1 1_02_00.mp4 1
Check match1 1_06_09.mp4 2
Check match1 1_02_02.mp4 1
Check match1 1_03_04.mp4 2
Check match1 1_06_08.mp4 1
Check match1 1_03_05.mp4 2
Check match1 1_06_06.mp4 2
Check match1 1_02_01.mp4 2
Check match2 1_00_02.mp4 1
Check match2 1_02_03.mp4 2
Check match2 1_06_09.mp4 2
Check match2 1_09_12.mp4 1
Check match2 1_08_12.mp4 2
Check match2 1_06_08.mp4 1
Check match2 1_08_11.mp4 1
Check match2 1_04_03.mp4 2
Check match3 1_01_00.mp4 2
Check match3 3_11_10.mp4 2
Check match3 2_04_07.mp4 2
Check match3 2_10_12.mp4 1
Check match3 1_08_10.mp4 2
Check match3 2_18_15.mp4 2
Check match3 1_12_17.mp4 2
Check match4 3_07_05.mp4 2
Check match4 2_14_17.mp4 2
Check match4 1_15_10.mp4 2
Check match4 2_02_05.mp4 2
Check match4 1_03_02.mp4 2
Check match4 2_05_07.mp4 2
Check match4 3_02_00.mp4 2
Check match4 3_18_17.mp4 2
Check match5 1_19_18.mp4 2
Check match5 2_20_17.mp4 1
C

# Create network to identify who hit the shuttle

In [4]:
import matplotlib.pyplot as plt
import tensorflow as tf
import math

from ai_badminton.trajectory import Trajectory
from ai_badminton.hit_detector import AdhocHitDetector, MLHitDetector
from ai_badminton.pose import Pose, read_player_poses, process_pose_file
from ai_badminton.court import Court, read_court
from ai_badminton.video_annotator import annotate_video

In [129]:
import os
import pandas as pd
import numpy as np
import cv2

def read_court(filename):
    file = open(filename, 'r')
    coordinates = [[float(x) for x in line.split(';')] for line in file]
    return coordinates

num_consec = 10
matches = list('match' + str(i) for i in range(1, 23))

x_train, y_train = [], []
for match in matches:
    for video in os.listdir('/home/code-base/scratch_space/data/%s/rally_video/' % match):
        if '.mp4' not in video:
            continue
        cap = cv2.VideoCapture('/home/code-base/scratch_space/data/%s/rally_video/%s' % (match, video))
        shape = cap.read()[1].shape
        height, width = shape[0], shape[1]

        rally, _ = os.path.splitext(video)
        trajectory = Trajectory('/home/code-base/scratch_space/data/%s/ball_trajectory/%s_ball.csv' % (match, rally))
#         trajectory = Trajectory('/home/code-base/scratch_space/data/%s/ball_trajectory/%s_ball.csv' % (match, rally), interp=False)
#         trajectory.X = np.array(trajectory.X)
#         trajectory.Y = np.array(trajectory.Y)
#         trajectory.X[trajectory.X != trajectory.X] = 0
#         trajectory.Y[trajectory.Y != trajectory.Y] = 0
#         trajectory.X = trajectory.X.tolist()
#         trajectory.Y = trajectory.Y.tolist()
        
        hit = pd.read_csv('/home/code-base/scratch_space/data/%s/shot/%s_hit.csv' % (match, rally))
        
        poses = read_player_poses('/home/code-base/scratch_space/data/%s/poses/%s' % (match, rally))
        bottom_player, top_player = poses[0], poses[1]
        
        x_list, y_list = [], []
        court_pts = read_court('/home/code-base/scratch_space/data/%s/court/%s.out' % (match, rally))
        corners = np.array([court_pts[1], court_pts[2], court_pts[0], court_pts[3]]).flatten()
    
        def reflect(x):
            x_ = np.array(x)
            for i in range(0, x.shape[1], 2):
                x_[:, i] = width - x_[:, i]
            return x_

        for i in range(num_consec):
            end = min(len(trajectory.X)-1, len(hit.hit)-1)-num_consec+i+1
            x_bird = np.array(list(zip(trajectory.X[i:end], trajectory.Y[i:end])))
            
            # Use entire pose
            x_pose = np.hstack([bottom_player.values[i:end], top_player.values[i:end]])
            x = np.hstack([x_bird, x_pose, np.array([corners for j in range(i, end)])])
        
            y = np.array(list(zip(trajectory.X[i+1:end+1], trajectory.Y[i+1:end+1])))
            x_list.append(x)
            y_list.append(y)

        x_t = np.hstack(x_list)
        y_t = y_list[-1]
        
        def scale_data(x):
            x = np.array(x)
            def scale_by_col(x, cols):
                x_ = x[:, cols]
                m, M = np.min(x_[x_ != 0]), np.max(x_[x_ != 0])
                x_[x_ != 0] = (x_[x_ != 0] - m) / (M - m) + 1
                x[:, cols] = x_
                return x
            
            even_cols = [2*i for i in range(x.shape[1] // 2)]
            odd_cols = [2*i+1 for i in range(x.shape[1] // 2)]
            x = scale_by_col(x, even_cols)
            x = scale_by_col(x, odd_cols)
            return x
          
        identity = lambda x: x
        def drop_data(x, p=0.15):
            x = np.array(x)
            # Corrupt 5% of the data
            indices = np.random.choice(
                np.arange(x.size), replace=False,
                size=int(x.size * p)
            )
            x[np.unravel_index(indices, x.shape)] = np.NaN
            df = pd.DataFrame(data=x)
            df.interpolate(method='slinear', inplace=True)
            df.fillna(method='bfill', inplace=True)
            df.fillna(method='ffill', inplace=True)
            x = df.values
            return x
        
        def jiggle_and_rotate(x):
            # Randomly shift by a vector in [-50, 50]
            # and rotate by a random amount between -10 and 10 degrees
            x = np.array(x.reshape((x.shape[0], x.shape[1] // 2, 2)))
            shift = (np.random.rand(1, 2) - 0.5) * 100
            angle = (np.random.rand() - 0.5) * math.pi / 180 * 20
            rotate = np.array([[math.cos(angle), math.sin(angle)], 
                               [math.sin(angle), -math.cos(angle)]])
            x = (x @ rotate) + shift
            x = x.reshape((x.shape[0], x.shape[1] * 2))
            return x
        
        drop_and_jiggle = lambda x: drop_data(jiggle_and_rotate(x))
        
        augmentations = [jiggle_and_rotate] * 3#[identity] + [jiggle_and_rotate] * 3
        for transform in augmentations:
#             Z = np.hstack([x_t, y_t])
#             S = scale_data(transform(Z))
#             x_train.append(np.array(S[:,:-2]))
#             y_train.append(np.array(S[:,-2:]))
            
#             S = scale_data(transform(reflect(Z)))
#             x_train.append(np.array(S[:,:-2]))
#             y_train.append(np.array(S[:,-2:]))
            
            Z = np.hstack([drop_data(x_t, p=0.3), y_t])
            S = scale_data(transform(Z))
            x_train.append(np.array(S[:,:-2]))
            y_train.append(np.array(S[:,-2:]))
            
            S = scale_data(transform(reflect(Z)))
            x_train.append(np.array(S[:,:-2]))
            y_train.append(np.array(S[:,-2:]))

x_train = np.vstack(x_train)
y_train = np.vstack(y_train)

match1 1_02_00 1 [13, 5]
match1 1_02_04 is hard [1, 1] [7059.074806800006, 859.8963510596025]
Manual label applied.
match1 1_06_08 1 [4, 0]
Manual label applied.
match1 1_02_02 1 [5, 1]
Manual label applied.
match1 1_06_06 2 [0, 5]
match1 1_03_05 2 [3, 10]
match1 1_02_01 2 [0, 6]
Manual label applied.
match1 1_03_04 2 [0, 3]
Manual label applied.
match1 1_06_09 1 [11, 3]
match1 1_02_03 is hard [1, 0] [5729.763825145301, 1e+99]
Manual label applied.
match1 1_03_06 2 [3, 5]
match1 1_01_00 2 [1, 9]
match2 1_00_02 1 [17, 1]
match2 1_08_11 1 [6, 1]
match2 1_06_08 1 [7, 0]
match2 1_09_12 1 [11, 0]
match2 1_08_12 1 [5, 1]
match2 1_04_03 2 [1, 5]
match2 1_06_09 2 [0, 10]
match2 1_02_03 2 [0, 5]
match3 2_04_07 2 [6, 15]
match3 2_18_15 2 [0, 37]
match3 1_12_17 2 [3, 5]
match3 3_11_10 2 [0, 6]
match3 2_10_12 1 [5, 1]
match3 1_08_10 is hard [1, 2] [4583.862849142903, 1176.3588272543989]
Manual label applied.
match3 1_01_00 2 [0, 13]
match4 2_14_17 1 [5, 3]
match4 3_07_05 1 [4, 1]
match4 3_02_00 2 

# Try other methods

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, test_size=0.15, shuffle=True)

from imblearn.over_sampling import SMOTE, ADASYN
sm = ADASYN()
x_resamp, y_resamp = sm.fit_resample(x_tr, y_tr)

In [380]:
rf_classifier = RandomForestClassifier(
    verbose=1, 
    n_jobs=16, 
#     class_weight='balanced',
    max_depth=20,
)
rf_classifier.fit(x_resamp, y_resamp)
y_pred = rf_classifier.predict(x_val)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:   18.3s


              precision    recall  f1-score   support

           0       1.00      0.95      0.97     16284
           1       0.80      0.98      0.88      1558
           2       0.78      0.97      0.86      1649

    accuracy                           0.95     19491
   macro avg       0.86      0.97      0.91     19491
weighted avg       0.96      0.95      0.96     19491

[[15471   365   448]
 [   22  1523    13]
 [   41     9  1599]]


[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:  1.1min finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.1s finished


In [386]:
from joblib import dump, load
os.chdir('/home/code-base/user_space/ai-badminton/TrackNetv2/3_in_3_out')
dump(rf_classifier, 'hitnet_rf.joblib')

['hitnet_rf.joblib']

In [8]:
os.getcwd()

'/trainman-mount/trainman-k8s-storage-6677a55e-8792-44ee-923d-9ed5edea05ca/ai-badminton/TrackNetv2/3_in_3_out'

In [148]:
# New hit detection
os.chdir('/home/code-base/user_space/ai-badminton/notebooks')

# Import shuttle trajectories and poses
root_name = 'anthony-ginting-beat-kento-momota-anthony-ginting-vs-kento-momota-fo-2019-hlzrivktbyo'

# Get poses and trajectories
trajectory = Trajectory('./data/%s_predict.csv' % root_name)

# Get video
cap = cv2.VideoCapture('./data/%s.mp4' % root_name)
if cap.isOpened() is False:
    print('Error opening video stream or file')

fps = cap.get(cv2.CAP_PROP_FPS)
_, image = cap.read()
size = (image.shape[1], image.shape[0])

court = Court(
    read_court('./data/%s_court.out' % root_name)
)
poses = read_player_poses('./output/%s' % root_name)

In [149]:
# trajectory.X = np.array(trajectory.X)
# trajectory.Y = np.array(trajectory.Y)
# trajectory.X[trajectory.X != trajectory.X] = 0
# trajectory.Y[trajectory.Y != trajectory.Y] = 0

In [150]:
def unscale_by_col(x, S, cols):
    x_ = x[:, cols]
    m, M = S[0], S[1]
    x_[x_ != 0] = (x_[x_ != 0] - 1) * (M - m) + m
    x[:, cols] = x_
    return x

def scale_by_col(x, cols):
    x_ = x[:, cols]
    m, M = np.min(x_[x_ != 0]), np.max(x_[x_ != 0])
    x_[x_ != 0] = (x_[x_ != 0] - m) / (M - m) + 1
    x[:, cols] = x_
    return x, m, M

def scale_data(x):
    even_cols = [2*i for i in range(x.shape[1] // 2)]
    odd_cols = [2*i+1 for i in range(x.shape[1] // 2)]
    x, me, Me = scale_by_col(x, even_cols)
    x, mo, Mo = scale_by_col(x, odd_cols)
    return x, (me, Me), (mo, Mo)

def unscale_data(x, Se, So):
    even_cols = [2*i for i in range(x.shape[1] // 2)]
    odd_cols = [2*i+1 for i in range(x.shape[1] // 2)]
    x = unscale_by_col(x, Se, even_cols)
    x = unscale_by_col(x, So, odd_cols)
    return x

Xb, Yb = trajectory.X, trajectory.Y
num_consec = int(model.input_shape[1] // (2 * (34 + 4 + 1)))
court_pts = court.corners
corner_coords = np.array([court_pts[1], court_pts[2], court_pts[0], court_pts[3]]).flatten()

bottom_player = poses[0]
top_player = poses[1]

corners = np.array([court_pts[1], court_pts[2], court_pts[0], court_pts[3]]).flatten()

x_list = []
L = min(bottom_player.values.shape[0], len(Xb))
for i in range(num_consec):
    end = L-num_consec+i+1
    x_bird = np.array(list(zip(Xb[i:end], Yb[i:end])))
    x_pose = np.hstack([bottom_player.values[i:end], top_player.values[i:end]])
    x = np.hstack([x_bird, x_pose, np.array([corners for j in range(i, end)])])

    x_list.append(x)
x_inp = np.hstack(x_list)
x_inp, Se, So = scale_data(x_inp)        

y_pred = model.predict(x_inp)
y_pred = unscale_data(y_pred, Se, So)

In [152]:
cap = cv2.VideoCapture('./data/%s.mp4' % root_name)
count = 0

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output/test.mp4', fourcc, fps, size)

L = 2000
for i in range(L):
    _, frame = cap.read()
    
    if i - 10 >= 0:
        pos = y_pred[i - 10]
        cv2.circle(frame, (int(pos[0]), int(pos[1])), 10, (0,0,255), -1)
    out.write(frame)
out.release()

In [151]:
y_pred

array([[434.88345, 337.66306],
       [464.05005, 242.06812],
       [474.89716, 209.3088 ],
       ...,
       [597.3514 , 367.27173],
       [600.15717, 326.5516 ],
       [584.9048 , 335.42133]], dtype=float32)

In [124]:
np.min(model.predict(x_inp))

0.7133125